In [1]:
import json
import subprocess
import sys

def ensure_package(package_name: str, install_name: str | None = None) -> None:
    """Install the pip package if it is missing from the current environment."""
    install_name = install_name or package_name
    result = subprocess.run(
        [sys.executable, "-m", "pip", "list", "--format=json"],
        check=True,
        capture_output=True,
        text=True,
    )
    installed = {pkg["name"].lower() for pkg in json.loads(result.stdout)}
    if package_name.lower() in installed:
        print(f"{package_name} already installed.")
        return
    subprocess.check_call([sys.executable, "-m", "pip", "install", install_name])
    print(f"Installed {install_name} as {package_name}.")

for package in [
    ("numpy", None),
    ("pandas", None),
    ("scikit-learn", "scikit-learn"),
    ("matplotlib", None),
    ("rdkit", "rdkit"),
]:
    ensure_package(*package)

numpy already installed.
pandas already installed.
scikit-learn already installed.
matplotlib already installed.
scikit-learn already installed.
matplotlib already installed.
rdkit already installed.
rdkit already installed.


In [2]:
import pandas as pd

data = pd.read_csv('../../../main-data/train.csv')

In [3]:
from pathlib import Path
from typing import Any, Dict, List, Optional, Set, Tuple

import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, Crippen, Descriptors, Descriptors3D, MACCSkeys, QED, rdMolDescriptors

feature_spec_path = Path("resource/melting_point_features_Version2(1).json")
with feature_spec_path.open() as fh:
    feature_spec: Dict[str, List[str]] = json.load(fh)
PRIMARY_FEATURES: List[str] = feature_spec["primary"]
SECONDARY_FEATURES: List[str] = feature_spec["secondary"]
ALL_FEATURES: List[str] = PRIMARY_FEATURES + SECONDARY_FEATURES

PERIODIC_TABLE = Chem.GetPeriodicTable()
PHENYL_PATTERN = Chem.MolFromSmarts("c1ccccc1")
FUNCTIONAL_GROUP_PATTERNS: Dict[str, Chem.Mol] = {
    "alcohol": Chem.MolFromSmarts("[CX4;!$(C=O)][OX2H]"),
    "amine": Chem.MolFromSmarts("[NX3;H2,H1;!$(NC=O);!$(N-O)]"),
    "amide": Chem.MolFromSmarts("[NX3][CX3](=[OX1])[#6]"),
    "carboxylic_acid": Chem.MolFromSmarts("C(=O)[OX2H1]"),
    "ester": Chem.MolFromSmarts("C(=O)O[#6]"),
    "ether": Chem.MolFromSmarts("[OD2]([#6])[#6]"),
    "thiol": Chem.MolFromSmarts("[#16X2H]"),
    "nitro": Chem.MolFromSmarts("[N+](=O)[O-]"),
    "sulfonyl": Chem.MolFromSmarts("[SX4](=O)(=O)[O,N,#6]"),
}
STRONG_DONOR_PATTERN = Chem.MolFromSmarts("[OX2H,NX3H,NX4H2+]")
STRONG_ACCEPTOR_PATTERN = Chem.MolFromSmarts("[O-]")
HALOGEN_ATOMIC_NUMBERS = {9, 17, 35, 53}

In [4]:
def canonicalize_dataset(df: pd.DataFrame) -> pd.DataFrame:
    """Add canonical SMILES, drop parse failures and duplicates."""
    df = df.copy()
    def to_canonical(smiles: str) -> Optional[str]:
        mol = Chem.MolFromSmiles(smiles)
        return Chem.MolToSmiles(mol, canonical=True) if mol else None
    df["canonical_smiles"] = df["SMILES"].map(to_canonical)
    before = len(df)
    df = df.dropna(subset=["canonical_smiles"]).drop_duplicates(subset=["canonical_smiles"]).reset_index(drop=True)
    print(f"Canonicalized SMILES: {before} -> {len(df)} unique molecules")
    return df



clean_data = canonicalize_dataset(data)

Canonicalized SMILES: 2662 -> 2660 unique molecules


In [5]:
from collections import deque


def count_substructures(mol: Chem.Mol, pattern: Optional[Chem.Mol]) -> int:
    if mol is None or pattern is None:
        return 0
    return len(mol.GetSubstructMatches(pattern))


def longest_aliphatic_chain_length(mol: Chem.Mol) -> int:
    aliphatic_indices = [
        atom.GetIdx()
        for atom in mol.GetAtoms()
        if atom.GetAtomicNum() == 6 and not atom.GetIsAromatic()
    ]
    if not aliphatic_indices:
        return 0
    adjacency: Dict[int, Set[int]] = {idx: set() for idx in aliphatic_indices}
    for bond in mol.GetBonds():
        begin, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
        if (
            begin in adjacency
            and end in adjacency
            and bond.GetBondType() == Chem.BondType.SINGLE
        ):
            adjacency[begin].add(end)
            adjacency[end].add(begin)
    if not any(adjacency.values()):
        return 1

    def bfs(start: int) -> Tuple[int, int]:
        visited = {start: 0}
        queue: deque[int] = deque([start])
        while queue:
            current = queue.popleft()
            for neighbor in adjacency[current]:
                if neighbor not in visited:
                    visited[neighbor] = visited[current] + 1
                    queue.append(neighbor)
        farthest = max(visited, key=visited.get)
        return farthest, visited[farthest]

    max_length = 1
    for idx in adjacency:
        farthest, _ = bfs(idx)
        _, dist = bfs(farthest)
        max_length = max(max_length, dist + 1)
    return max_length


def embed_molecule_3d(mol_with_h: Chem.Mol) -> Optional[Chem.Mol]:
    mol3d = Chem.Mol(mol_with_h)
    params = AllChem.ETKDGv3() if hasattr(AllChem, "ETKDGv3") else AllChem.ETKDG()
    params.randomSeed = 0xF00D
    try:
        status = AllChem.EmbedMolecule(mol3d, params)
        if status != 0:
            return None
        AllChem.UFFOptimizeMolecule(mol3d, maxIters=200)
    except ValueError:
        return None
    return mol3d


def extract_gasteiger_charges(mol_with_h: Chem.Mol) -> List[float]:
    charges: List[float] = []
    try:
        AllChem.ComputeGasteigerCharges(mol_with_h)
    except Exception:
        return [np.nan] * mol_with_h.GetNumAtoms()
    for atom in mol_with_h.GetAtoms():
        charge_val = atom.GetProp("_GasteigerCharge") if atom.HasProp("_GasteigerCharge") else "nan"
        try:
            charge = float(charge_val)
        except ValueError:
            charge = np.nan
        charges.append(charge)
    return charges

In [6]:
def compute_features_from_smiles(smiles: str) -> Tuple[Optional[Dict[str, Any]], Dict[str, Any]]:
    meta: Dict[str, Any] = {"smiles": smiles, "canonical_smiles": None, "error": None, "missing": []}
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        meta["error"] = "smiles_parse_failed"
        return None, meta

    canonical = Chem.MolToSmiles(mol, canonical=True)
    meta["canonical_smiles"] = canonical

    feature_values: Dict[str, Any] = {name: np.nan for name in ALL_FEATURES}
    feature_values["canonical_smiles"] = canonical
    for name in ("gasteiger_charges", "principal_moments_3d", "morgan_fingerprint_bits", "maccs_keys_bits"):
        feature_values[name] = None

    num_atoms = mol.GetNumAtoms()
    num_bonds = mol.GetNumBonds()
    ring_info = mol.GetRingInfo()
    num_rings = ring_info.NumRings() if ring_info is not None else 0
    num_ring_atoms = sum(1 for atom in mol.GetAtoms() if atom.IsInRing())
    aromatic_atom_count = sum(1 for atom in mol.GetAtoms() if atom.GetIsAromatic())

    mol_with_h = Chem.AddHs(mol)
    charges = extract_gasteiger_charges(mol_with_h)
    mol3d = embed_molecule_3d(mol_with_h)

    # Core counts
    feature_values["num_atoms"] = num_atoms
    feature_values["num_heavy_atoms"] = mol.GetNumHeavyAtoms()
    feature_values["num_hydrogens"] = sum(atom.GetTotalNumHs(includeNeighbors=True) for atom in mol.GetAtoms())
    feature_values["num_heteroatoms"] = sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() not in (1, 6))
    feature_values["num_valence_electrons"] = sum(PERIODIC_TABLE.GetNOuterElecs(atom.GetAtomicNum()) for atom in mol.GetAtoms())

    feature_values["num_rings"] = num_rings
    calc_num_aromatic_rings = getattr(rdMolDescriptors, "CalcNumAromaticRings", None)
    feature_values["aromatic_ring_count"] = calc_num_aromatic_rings(mol) if calc_num_aromatic_rings else Descriptors.NumAromaticRings(mol)
    calc_num_aliphatic_rings = getattr(rdMolDescriptors, "CalcNumAliphaticRings", None)
    feature_values["aliphatic_ring_count"] = calc_num_aliphatic_rings(mol) if calc_num_aliphatic_rings else 0
    feature_values["largest_ring_size"] = max((len(r) for r in Chem.GetSymmSSSR(mol)), default=0)
    calc_num_ring_systems = getattr(rdMolDescriptors, "CalcNumRingSystems", None)
    num_ring_systems = calc_num_ring_systems(mol) if calc_num_ring_systems else num_rings
    feature_values["num_ring_systems"] = num_ring_systems
    calc_num_spiro_atoms = getattr(rdMolDescriptors, "CalcNumSpiroAtoms", None)
    feature_values["num_spiro_atoms"] = calc_num_spiro_atoms(mol) if calc_num_spiro_atoms else 0
    feature_values["num_fused_rings"] = max(num_rings - num_ring_systems, 0)
    feature_values["has_aromatic_ring"] = int(feature_values["aromatic_ring_count"] > 0)
    feature_values["has_phenyl_group"] = int(mol.HasSubstructMatch(PHENYL_PATTERN))

    feature_values["num_double_bonds"] = sum(1 for bond in mol.GetBonds() if bond.GetBondType() == Chem.BondType.DOUBLE)
    feature_values["num_triple_bonds"] = sum(1 for bond in mol.GetBonds() if bond.GetBondType() == Chem.BondType.TRIPLE)
    feature_values["num_conjugated_bonds"] = sum(1 for bond in mol.GetBonds() if bond.GetIsConjugated())
    unsaturated_bonds = sum(1 for bond in mol.GetBonds() if bond.GetIsAromatic() or bond.GetBondType() != Chem.BondType.SINGLE)
    feature_values["saturation_index"] = unsaturated_bonds / num_bonds if num_bonds else 0.0

    calc_rotatable_bonds = getattr(rdMolDescriptors, "CalcNumRotatableBonds", None)
    feature_values["num_rotatable_bonds"] = calc_rotatable_bonds(mol) if calc_rotatable_bonds else Descriptors.NumRotatableBonds(mol)
    branching_atoms = sum(1 for atom in mol.GetAtoms() if atom.GetDegree() > 2)
    feature_values["degree_of_branchedness"] = branching_atoms / num_atoms if num_atoms else 0.0
    feature_values["topological_complexity"] = Descriptors.BertzCT(mol)
    feature_values["longest_aliphatic_chain_length"] = longest_aliphatic_chain_length(mol)

    distance_matrix = Chem.rdmolops.GetDistanceMatrix(mol)
    if distance_matrix.size:
        eccentricities = distance_matrix.max(axis=1)
        feature_values["topological_diameter"] = float(distance_matrix.max())
        feature_values["topological_radius"] = float(eccentricities.min())
    else:
        feature_values["topological_diameter"] = 0.0
        feature_values["topological_radius"] = 0.0

    # Functional group presence/counts
    group_counts = {name: count_substructures(mol, pattern) for name, pattern in FUNCTIONAL_GROUP_PATTERNS.items()}
    feature_values["has_alcohol"] = int(group_counts["alcohol"] > 0)
    feature_values["has_amine"] = int(group_counts["amine"] > 0)
    feature_values["has_amide"] = int(group_counts["amide"] > 0)
    feature_values["has_carboxylic_acid"] = int(group_counts["carboxylic_acid"] > 0)
    feature_values["has_ester"] = int(group_counts["ester"] > 0)
    feature_values["has_ether"] = int(group_counts["ether"] > 0)
    feature_values["has_halogen"] = int(any(atom.GetAtomicNum() in HALOGEN_ATOMIC_NUMBERS for atom in mol.GetAtoms()))
    feature_values["has_thiol"] = int(group_counts["thiol"] > 0)
    feature_values["has_nitro"] = int(group_counts["nitro"] > 0)
    feature_values["has_sulfonyl"] = int(group_counts["sulfonyl"] > 0)

    feature_values["num_alcohols"] = group_counts["alcohol"]
    feature_values["num_amines"] = group_counts["amine"]
    feature_values["num_amides"] = group_counts["amide"]
    feature_values["num_carboxylic_acids"] = group_counts["carboxylic_acid"]
    feature_values["num_esters"] = group_counts["ester"]
    feature_values["num_ethers"] = group_counts["ether"]
    feature_values["num_thiols"] = group_counts["thiol"]
    feature_values["num_nitro_groups"] = group_counts["nitro"]
    feature_values["num_sulfonyl_groups"] = group_counts["sulfonyl"]

    feature_values["num_halogen_atoms"] = sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() in HALOGEN_ATOMIC_NUMBERS)
    feature_values["num_fluorine_atoms"] = sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() == 9)
    feature_values["num_chlorine_atoms"] = sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() == 17)
    feature_values["num_bromine_atoms"] = sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() == 35)
    feature_values["num_iodine_atoms"] = sum(1 for atom in mol.GetAtoms() if atom.GetAtomicNum() == 53)

    calc_num_aromatic_heterocycles = getattr(rdMolDescriptors, "CalcNumAromaticHeterocycles", None)
    feature_values["num_aromatic_heterocycles"] = calc_num_aromatic_heterocycles(mol) if calc_num_aromatic_heterocycles else 0
    calc_num_aromatic_carbocycles = getattr(rdMolDescriptors, "CalcNumAromaticCarbocycles", None)
    feature_values["num_aromatic_carbocycles"] = calc_num_aromatic_carbocycles(mol) if calc_num_aromatic_carbocycles else 0

    calc_hbd = getattr(rdMolDescriptors, "CalcNumLipinskiHBD", None)
    calc_hba = getattr(rdMolDescriptors, "CalcNumLipinskiHBA", None)
    feature_values["hbond_donor_count"] = calc_hbd(mol) if calc_hbd else Descriptors.NumHDonors(mol)
    feature_values["hbond_acceptor_count"] = calc_hba(mol) if calc_hba else Descriptors.NumHAcceptors(mol)
    feature_values["num_strong_hbond_donors"] = count_substructures(mol, STRONG_DONOR_PATTERN)
    feature_values["num_strong_hbond_acceptors"] = count_substructures(mol, STRONG_ACCEPTOR_PATTERN)

    formal_charge = sum(atom.GetFormalCharge() for atom in mol.GetAtoms())
    feature_values["formal_charge"] = formal_charge
    feature_values["num_positive_atoms"] = sum(1 for atom in mol.GetAtoms() if atom.GetFormalCharge() > 0)
    feature_values["num_negative_atoms"] = sum(1 for atom in mol.GetAtoms() if atom.GetFormalCharge() < 0)
    feature_values["is_zwitterionic"] = int(feature_values["num_positive_atoms"] > 0 and feature_values["num_negative_atoms"] > 0)

    feature_values["molecular_weight"] = Descriptors.MolWt(mol)
    feature_values["exact_molecular_weight"] = rdMolDescriptors.CalcExactMolWt(mol)
    calc_fraction_csp3 = getattr(rdMolDescriptors, "CalcFractionCSP3", None)
    feature_values["fraction_csp3"] = calc_fraction_csp3(mol) if calc_fraction_csp3 else Descriptors.FractionCSP3(mol)
    feature_values["aromatic_proportion"] = aromatic_atom_count / num_atoms if num_atoms else 0.0

    feature_values["logp"] = Crippen.MolLogP(mol)
    feature_values["tpsa"] = rdMolDescriptors.CalcTPSA(mol)
    feature_values["mol_refractivity"] = Crippen.MolMR(mol)
    try:
        feature_values["qed_weighted"] = QED.qed(mol)
    except Exception:
        feature_values["qed_weighted"] = np.nan

    # Extra topological indices (chi/kappa)
    try:
        feature_values["chi0v"] = rdMolDescriptors.CalcChi0v(mol)
        feature_values["chi1v"] = rdMolDescriptors.CalcChi1v(mol)
        feature_values["chi0n"] = rdMolDescriptors.CalcChi0n(mol)
        feature_values["chi1n"] = rdMolDescriptors.CalcChi1n(mol)
        feature_values["kappa1"] = rdMolDescriptors.CalcKappa1(mol)
        feature_values["kappa2"] = rdMolDescriptors.CalcKappa2(mol)
        feature_values["kappa3"] = rdMolDescriptors.CalcKappa3(mol)
    except Exception:
        pass

    charges_array = np.array(charges, dtype=float)
    valid_charge_mask = ~np.isnan(charges_array)
    if valid_charge_mask.any():
        valid_charges = charges_array[valid_charge_mask]
        feature_values["gasteiger_charges"] = tuple(np.round(charges_array, 6))
        feature_values["gasteiger_max_partial_charge"] = float(valid_charges.max())
        feature_values["gasteiger_min_partial_charge"] = float(valid_charges.min())
        feature_values["gasteiger_sum_absolute_charges"] = float(np.abs(valid_charges).sum())
        feature_values["gasteiger_charge_std_dev"] = float(valid_charges.std(ddof=0))
        feature_values["gasteiger_charge_range"] = float(valid_charges.max() - valid_charges.min())
    else:
        feature_values["gasteiger_charges"] = None

    feature_values["morgan_fingerprint_bits"] = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048).ToBitString()
    feature_values["maccs_keys_bits"] = MACCSkeys.GenMACCSKeys(mol).ToBitString()

    calc_atom_stereo = getattr(rdMolDescriptors, "CalcNumAtomStereoCenters", None)
    calc_unspecified_atom_stereo = getattr(rdMolDescriptors, "CalcNumUnspecifiedAtomStereoCenters", None)
    total_atom_stereo = calc_atom_stereo(mol) if calc_atom_stereo else 0
    unspecified_atom_stereo = calc_unspecified_atom_stereo(mol) if calc_unspecified_atom_stereo else 0
    calc_num_bond_stereo = getattr(rdMolDescriptors, "CalcNumBondStereoCenters", None)
    calc_num_unspecified_bond_stereo = getattr(rdMolDescriptors, "CalcNumUnspecifiedBondStereoCenters", None)
    total_bond_stereo = calc_num_bond_stereo(mol) if calc_num_bond_stereo else 0
    unspecified_bond_stereo = calc_num_unspecified_bond_stereo(mol) if calc_num_unspecified_bond_stereo else 0
    feature_values["num_stereo_centers"] = total_atom_stereo + total_bond_stereo
    feature_values["num_defined_stereo_centers"] = (total_atom_stereo - unspecified_atom_stereo) + (total_bond_stereo - unspecified_bond_stereo)
    feature_values["num_undefined_stereo_centers"] = unspecified_atom_stereo + unspecified_bond_stereo

    # Secondary metrics derived from existing values
    feature_values["ring_atom_fraction"] = num_ring_atoms / num_atoms if num_atoms else 0.0
    feature_values["aromatic_atom_fraction"] = aromatic_atom_count / num_atoms if num_atoms else 0.0
    feature_values["heteroatom_fraction"] = feature_values["num_heteroatoms"] / num_atoms if num_atoms else 0.0
    feature_values["rotatable_bond_fraction"] = feature_values["num_rotatable_bonds"] / num_bonds if num_bonds else 0.0
    heavy_atom_count = feature_values["num_heavy_atoms"] or 1
    feature_values["hbond_donor_density"] = feature_values["hbond_donor_count"] / heavy_atom_count
    feature_values["hbond_acceptor_density"] = feature_values["hbond_acceptor_count"] / heavy_atom_count
    feature_values["polarity_to_size_ratio"] = feature_values["tpsa"] / feature_values["molecular_weight"] if feature_values["molecular_weight"] else np.nan
    feature_values["rigidity_index"] = feature_values["num_rings"] / (feature_values["num_rotatable_bonds"] + 1)
    try:
        estate_indices = np.array(rdMolDescriptors.CalcEStateIndices(mol), dtype=float)
        feature_values["estate_sum"] = float(estate_indices.sum())
        feature_values["estate_mean"] = float(estate_indices.mean())
        feature_values["estate_max"] = float(estate_indices.max())
        feature_values["estate_min"] = float(estate_indices.min())
    except Exception:
        pass

    fragments = Chem.GetMolFrags(mol, asMols=True)
    if fragments:
        feature_values["num_aromatic_rings_fragments"] = sum((calc_num_aromatic_rings(frag) if calc_num_aromatic_rings else Descriptors.NumAromaticRings(frag)) for frag in fragments)
        feature_values["num_heteroatom_fragments"] = sum(1 for frag in fragments if any(atom.GetAtomicNum() not in (1, 6) for atom in frag.GetAtoms()))
        if calc_hbd and calc_hba:
            feature_values["num_hbonding_fragments"] = sum(1 for frag in fragments if calc_hbd(frag) + calc_hba(frag) > 0)
        else:
            feature_values["num_hbonding_fragments"] = sum(1 for frag in fragments if Descriptors.NumHDonors(frag) + Descriptors.NumHAcceptors(frag) > 0)
    else:
        feature_values["num_aromatic_rings_fragments"] = 0
        feature_values["num_heteroatom_fragments"] = 0
        feature_values["num_hbonding_fragments"] = 0

    # 3D features (only if embedding succeeded)
    if mol3d is not None and mol3d.GetNumConformers() > 0:
        feature_values["conformer_count"] = mol3d.GetNumConformers()
        calc_labute_asa = getattr(rdMolDescriptors, "CalcLabuteASA", None)
        feature_values["surface_area_3d"] = calc_labute_asa(mol3d) if calc_labute_asa else np.nan
        calc_mol_volume = getattr(rdMolDescriptors, "CalcMolVolume", None)
        feature_values["volume_3d"] = calc_mol_volume(mol3d) if calc_mol_volume else np.nan
        pmi1 = Descriptors3D.PMI1(mol3d)
        pmi2 = Descriptors3D.PMI2(mol3d)
        pmi3 = Descriptors3D.PMI3(mol3d)
        feature_values["principal_moments_3d"] = (pmi1, pmi2, pmi3)
        feature_values["radius_of_gyration_3d"] = Descriptors3D.RadiusOfGyration(mol3d)
        feature_values["inertia_mass_3d"] = pmi1 + pmi2 + pmi3
        feature_values["asphericity_3d"] = Descriptors3D.Asphericity(mol3d)
        feature_values["eccentricity_3d"] = Descriptors3D.Eccentricity(mol3d)
        feature_values["inertial_shape_factor_3d"] = Descriptors3D.InertialShapeFactor(mol3d)
        calc_pbf = getattr(rdMolDescriptors, "CalcPBF", None)
        feature_values["planarity_proxy_index"] = calc_pbf(mol3d) if calc_pbf else np.nan
        if feature_values["surface_area_3d"] and feature_values["volume_3d"]:
            feature_values["surface_area_to_volume_ratio"] = feature_values["surface_area_3d"] / feature_values["volume_3d"]

        if valid_charge_mask.any():
            coords = mol3d.GetConformer().GetPositions()
            centroid = coords.mean(axis=0)
            charge_distance = np.array([charges_array[i] * np.linalg.norm(coords[i] - centroid) for i in range(len(charges_array)) if not np.isnan(charges_array[i])], dtype=float)
            if charge_distance.size:
                feature_values["dipole_proxy_range_3d"] = float(charge_distance.max() - charge_distance.min())
                feature_values["dipole_proxy_spread_3d"] = float(charge_distance.std(ddof=0))
    else:
        meta["missing"].extend([
            "conformer_count",
            "surface_area_3d",
            "volume_3d",
            "principal_moments_3d",
            "radius_of_gyration_3d",
            "inertia_mass_3d",
            "asphericity_3d",
            "eccentricity_3d",
            "inertial_shape_factor_3d",
            "surface_area_to_volume_ratio",
            "dipole_proxy_range_3d",
            "dipole_proxy_spread_3d",
            "planarity_proxy_index",
        ])

    if "surface_area_to_volume_ratio" not in feature_values:
        feature_values["surface_area_to_volume_ratio"] = np.nan

    meta["missing"].extend(
        name for name, value in feature_values.items() if value is None or (isinstance(value, float) and np.isnan(value))
    )
    meta["missing"] = sorted(set(meta["missing"]))

    return feature_values, meta

In [7]:
feature_records: List[Dict[str, Any]] = []
metadata_records: List[Dict[str, Any]] = []
for row in clean_data.itertuples(index=False):
    features, meta = compute_features_from_smiles(row.SMILES)
    meta["id"] = row.id
    metadata_records.append(meta)
    if features is None:
        continue
    features["id"] = row.id
    feature_records.append(features)

feature_df = pd.DataFrame(feature_records).set_index("id")
base_cols = ["canonical_smiles"] + ALL_FEATURES
extra_cols = [c for c in feature_df.columns if c not in base_cols]
feature_df = feature_df.reindex(columns=base_cols + sorted(extra_cols))
metadata_df = pd.DataFrame(metadata_records).set_index("id")

print(f"Extracted features for {len(feature_df)} molecules; {metadata_df['error'].notna().sum()} failures.")
feature_df.head()

[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerator
[05:18:24] DEPRECATION WARNING: please use MorganGenerat

Extracted features for 2660 molecules; 0 failures.


[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerator
[05:18:48] DEPRECATION WARNING: please use MorganGenerat

,canonical_smiles,num_atoms,num_heavy_atoms,num_hydrogens,num_heteroatoms,num_valence_electrons,num_rings,aromatic_ring_count,aliphatic_ring_count,largest_ring_size,...,num_hbonding_fragments,chi0n,chi0v,chi1n,chi1v,kappa1,kappa2,kappa3,mol_refractivity,qed_weighted
id,,,,,,,,,,,,,,,,,,,,,
2175,FC1=C(F)C(F)(F)C1(F)F,10,10,0,6,58,1,0,1,4,...,0,4.267787,4.267787,2.133893,2.133893,7.427296,1.486106,0.494331,19.1360,0.479530
1222,c1ccc2c(c1)ccc1[nH]c3ccccc3c12,17,17,11,1,69,4,4,0,6,...,1,9.273503,9.273503,5.815384,5.815384,8.824469,3.109248,1.136564,73.3107,0.446879
2994,CCn1c(C)nc2ccccc21,12,12,12,2,50,2,2,0,6,...,1,7.410935,7.410935,4.245112,4.245112,7.345075,2.626943,0.915677,50.1440,0.625839
1704,CC#CC(=O)O,6,6,4,2,28,0,0,0,0,...,1,3.355462,3.355462,1.427731,1.427731,5.030000,2.278139,3.030000,21.0858,0.422302
2526,CCCCC(C)S,7,7,14,1,30,0,0,0,0,...,0,5.145884,6.040311,2.950904,3.467302,7.350000,4.507480,6.350000,37.9730,0.540734


In [8]:
missing_counts = (
    metadata_df["missing"].explode().dropna().value_counts().sort_values(ascending=False)
)
missing_counts.head(20)

missing
surface_area_to_volume_ratio    2660
volume_3d                       2660
asphericity_3d                     2
conformer_count                    2
dipole_proxy_range_3d              2
dipole_proxy_spread_3d             2
eccentricity_3d                    2
inertia_mass_3d                    2
inertial_shape_factor_3d           2
planarity_proxy_index              2
principal_moments_3d               2
radius_of_gyration_3d              2
surface_area_3d                    2
Name: count, dtype: int64

In [9]:
targets = data.set_index("id")["Tm"]
feature_df["Tm"] = targets

In [10]:
output_dir = Path("result/data")
output_dir.mkdir(parents=True, exist_ok=True)
parquet_path = output_dir / "melting_point_features.parquet"
csv_path = output_dir / "melting_point_features.csv"

try:
    feature_df.to_parquet(parquet_path)
    parquet_note = f"Saved parquet to {parquet_path}."
except Exception as exc:
    parquet_note = f"Skipped parquet export ({exc})."
feature_df.to_csv(csv_path)
print(parquet_note)
print(f"Saved CSV to {csv_path}.")

Skipped parquet export (Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.).
Saved CSV to result/data/melting_point_features.csv.
